# Text processing and text-based feature eng/extraction

Regular env preparations.

In [1]:
import os
import os.path
import pandas as pd
import datetime
import matplotlib
import pickle
import numpy as np
import pandas as pd
from polyglot.text import Text
# !polyglot download sentiment2.ru
import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger_ru')
# nltk.download('tagsets')
import re

ImportError: No module named 'polyglot'

In [443]:
matplotlib.rcParams['figure.figsize'] = (14.0, 7.0)
pd.set_option('display.max_columns', 100)

In [444]:
DATA_PATH = 'D://DSworkshop//Avito'

In [445]:
all_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))

In [446]:
train = pd.DataFrame(all_train[1000000:])

Let's see some data!

In [447]:
train.head(10)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
1000000,cf348255c0c1,4e3cfbb60abc,Волгоградская область,Волгоград,Для дома и дачи,Мебель и интерьер,Шкафы и комоды,NaN,NaN,Витрина-шкаф стеклянная импортная,Б/у хорош.,5500.0,77,2017-03-19,Company,326215e9b350e828d4bf440c7c7d92018ad1d808b3d45a...,1511.0,0.86521
1000001,cac8c3840cdc,bc454d4dc63c,Ставропольский край,Ставрополь,Личные вещи,Товары для детей и игрушки,Игрушки,NaN,NaN,Неваляшеа,Неваляшка новая 28 см,400.0,49,2017-03-23,Private,ae12ed2c0db0226919f7a9ff2512e26104e78e8722fbcd...,703.0,0.00000
1000002,008bb9e305b5,6d04035370f7,Нижегородская область,Нижний Новгород,Бытовая электроника,Товары для компьютера,Комплектующие,Видеокарты,NaN,Видеокарта asus geforce 210 (Новая),Интерфейс: PCI-E x16/\nТип: nVidia GeForce 210...,1000.0,4,2017-03-23,Private,0fd0580ee76984f30d9d498f17dfe13b289e1d7c95f608...,2865.0,0.76786
1000003,4b05d6d213ed,68a3f4a6c928,Краснодарский край,Краснодар,Личные вещи,Часы и украшения,Ювелирные изделия,NaN,NaN,Кольцо из красного золота 585 пр,"Золотое кольцо 585 пр./\n /\nВес - 5,61 гр./\n...",12000.0,12276,2017-03-22,Shop,2a413ffdd5d746c54d51c2fd6fb0c4fb924a4010210278...,25.0,0.00000
1000004,1fc917874c95,fe44d46f5b5f,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для мальчиков,Другое,NaN,Праздничный костюм,продам праздничный костюм тройка. состояние от...,500.0,12,2017-03-18,Private,e6cb46627f268887bb0966aeff067c32ab219074b61dc0...,76.0,0.00000
1000005,5d4eb400d31b,eeff3e5094af,Саратовская область,Саратов,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),Платье,Платье отношено 1 сезон/\nСтало маловато(((,500.0,36,2017-03-26,Private,7c9bbf9f344fef4a65e3b6817a6c9eff5b1bc22052b1ef...,387.0,0.00000
1000006,006267a1c422,35470a71a5a4,Тюменская область,Тюмень,Для дома и дачи,Мебель и интерьер,Шкафы и комоды,NaN,NaN,Шкаф-купе 3-х створчатый,Материал: ЛДСП/\n /\nГабаритные размеры: 1510*...,8330.0,44,2017-03-15,Shop,6066ec0ae2c0522945a5bb090306657d0bae3a0947fd24...,1487.0,0.00000
1000007,859ee807cbbb,0a33877cefcc,Удмуртия,Ижевск,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,86-92 см (1-2 года),Зимний костюм рост до 86 см,"продам зимний костюм рост до 86 см, состояние ...",350.0,80,2017-03-23,Private,6a48c736920310a38550c87701eb77a7be0d8a63bf5c6c...,96.0,0.00000
1000008,3bcbed7a74b0,5efcdee6890d,Челябинская область,Троицк,Для дома и дачи,Ремонт и строительство,Стройматериалы,NaN,NaN,Стойматериалы,продам б\у стройматериал .плита ПТК 59-60 цена...,NaN,5,2017-03-15,Private,NaN,NaN,0.27785
1000009,e74b45079aad,eb40090bd40b,Калининградская область,Калининград,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),Новое платье трапеция,Новое платье трапеция. Отличное качество. Вста...,1300.0,109,2017-03-18,Private,dfce005963fa6034312de8ffcf4d545d9570a7590b87f9...,572.0,0.00000


How many unique categories and parent categories do we have?

In [448]:
print ("Num of parent categories: ",len(train["parent_category_name"].unique()))
print ("Num of categories: ",len(train["category_name"].unique()))

Num of parent categories:  9
Num of categories:  47


How many unique (distinct) "param"s we have?

In [449]:
print("Num of distinct params: "
      ,len(set(train["param_1"].tolist()) | set(train["param_2"].tolist()) | set(train["param_3"].tolist())))

Num of distinct params:  1572


Nice. We think that combining those with the "params" attributes can generate the most informative features about of the data. But in order to engeneer those, we need to "categorize" it somehow. Lets see how many ads do not have any param mentioned within it.

In [450]:
ads_with_absent_params = len(train[train["param_1"].isnull() & train["param_2"].isnull() & train["param_3"].isnull()])
total_ads = train.shape[0]
print ("Absent params:",ads_with_absent_params)
print ("Total ads:",total_ads)
print ("Ratio of ads with no params at all:",100*ads_with_absent_params / float(total_ads),"%")

Absent params: 20850
Total ads: 503424
Ratio of ads with no params at all: 4.141638062547673 %


## Basic text feature extraction

### Text length

Word counts for text features.

In [451]:
def merge_params(row):
    sentence = ""
    for att in ["param_1","param_2","param_3"]:
        if not pd.isnull(row[att]):
            sentence += row[att] + " "
    return sentence

train["merged_params"] = ""
for att in ["param_1","param_2","param_3"]:
    train["merged_params"] = np.where(~train[att].isnull(), train["merged_params"].astype(str) + " " + train[att], train["merged_params"])

for att in ["title", "description", "merged_params"]:
    train[att].fillna("", inplace=True)
    train[att + "_word_count"] = train[att].str.split().apply(len)

display(train.head(5))

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,merged_params,title_word_count,description_word_count,merged_params_word_count
1000000,cf348255c0c1,4e3cfbb60abc,Волгоградская область,Волгоград,Для дома и дачи,Мебель и интерьер,Шкафы и комоды,NaN,NaN,Витрина-шкаф стеклянная импортная,Б/у хорош.,5500.0,77,2017-03-19,Company,326215e9b350e828d4bf440c7c7d92018ad1d808b3d45a...,1511.0,0.86521,Шкафы и комоды,3,2,3
1000001,cac8c3840cdc,bc454d4dc63c,Ставропольский край,Ставрополь,Личные вещи,Товары для детей и игрушки,Игрушки,NaN,NaN,Неваляшеа,Неваляшка новая 28 см,400.0,49,2017-03-23,Private,ae12ed2c0db0226919f7a9ff2512e26104e78e8722fbcd...,703.0,0.00000,Игрушки,1,4,1
1000002,008bb9e305b5,6d04035370f7,Нижегородская область,Нижний Новгород,Бытовая электроника,Товары для компьютера,Комплектующие,Видеокарты,NaN,Видеокарта asus geforce 210 (Новая),Интерфейс: PCI-E x16/\nТип: nVidia GeForce 210...,1000.0,4,2017-03-23,Private,0fd0580ee76984f30d9d498f17dfe13b289e1d7c95f608...,2865.0,0.76786,Комплектующие Видеокарты,5,55,2
1000003,4b05d6d213ed,68a3f4a6c928,Краснодарский край,Краснодар,Личные вещи,Часы и украшения,Ювелирные изделия,NaN,NaN,Кольцо из красного золота 585 пр,"Золотое кольцо 585 пр./\n /\nВес - 5,61 гр./\n...",12000.0,12276,2017-03-22,Shop,2a413ffdd5d746c54d51c2fd6fb0c4fb924a4010210278...,25.0,0.00000,Ювелирные изделия,6,35,2
1000004,1fc917874c95,fe44d46f5b5f,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для мальчиков,Другое,NaN,Праздничный костюм,продам праздничный костюм тройка. состояние от...,500.0,12,2017-03-18,Private,e6cb46627f268887bb0966aeff067c32ab219074b61dc0...,76.0,0.00000,Для мальчиков Другое,2,19,3


In [452]:
train.describe()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count
count,4.747930e+05,503424.000000,465718.000000,503424.000000,503424.000000,503424.000000,503424.000000
mean,4.270890e+05,757.584619,1242.017173,0.139364,3.324291,25.126468,3.590051
std,1.154513e+08,5710.284645,970.004098,0.260807,1.844069,40.209250,2.323999
min,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,5.000000e+02,9.000000,425.000000,0.000000,2.000000,6.000000,2.000000
50%,1.300000e+03,29.000000,1057.000000,0.000000,3.000000,12.000000,3.000000
75%,7.000000e+03,89.000000,2217.000000,0.149890,4.000000,26.000000,5.000000
max,7.950101e+10,204429.000000,3066.000000,1.000000,21.000000,711.000000,9.000000


Sentence count for titles and descriptions.

In [453]:
train['description_sentence_count'] = train.apply(lambda row: len(Text(row['description']).sentences) if len(row['description']) > 0 else 0, axis=1)

Word to sentence ratio.

In [454]:
train['description_words/sentence_ratio'] = train["description_word_count"] / train["description_sentence_count"]

In [455]:
train.describe()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio
count,4.747930e+05,503424.000000,465718.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,464619.000000
mean,4.270890e+05,757.584619,1242.017173,0.139364,3.324291,25.126468,3.590051,4.678508,7.050668
std,1.154513e+08,5710.284645,970.004098,0.260807,1.844069,40.209250,2.323999,9.087564,6.145197
min,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.300000
25%,5.000000e+02,9.000000,425.000000,0.000000,2.000000,6.000000,2.000000,1.000000,3.666667
50%,1.300000e+03,29.000000,1057.000000,0.000000,3.000000,12.000000,3.000000,2.000000,5.400000
75%,7.000000e+03,89.000000,2217.000000,0.149890,4.000000,26.000000,5.000000,4.000000,8.300000
max,7.950101e+10,204429.000000,3066.000000,1.000000,21.000000,711.000000,9.000000,352.000000,273.000000


### Capital letters and other chars in text columns

In [456]:
def count_capitals_ratio_safe(s):
    if pd.isnull(s) or len(s)==0:
        return 0
    s = s.replace(" ","")
    return len([l for l in s if l.isupper()])/len(s)

train['title_capital_letters_ratio'] = train.apply(lambda row:count_capitals_ratio_safe(row['title']), axis=1)
train['description_capital_letters_ratio'] = train.apply(lambda row:count_capitals_ratio_safe(row['description']), axis=1)

In [457]:
def count_non_regular_chars_ratio_safe(s):
    if pd.isnull(s) or len(s)==0:
        return 0
    s = s.replace(" ","")
    pat = re.compile("[\W]")
    return len([l for l in s if pat.match(l)])/len(s)

train['title_non_regular_chars_ratio'] = train.apply(lambda row:count_non_regular_chars_ratio_safe(row['title']), axis=1)
train['description_non_regular_chars_ratio'] = train.apply(lambda row:count_non_regular_chars_ratio_safe(row['description']), axis=1)

In [458]:
train['title_num_of_newrow_char'] = train.apply(lambda row:row['title'].count('\n'), axis=1)
train['description_num_of_newrow_char'] = train.apply(lambda row:row['description'].count('\n'), axis=1)

In [459]:
train.describe()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char
count,4.747930e+05,503424.000000,465718.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,464619.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.0,503424.000000
mean,4.270890e+05,757.584619,1242.017173,0.139364,3.324291,25.126468,3.590051,4.678508,7.050668,0.097784,0.048276,0.033604,0.062609,0.0,1.885049
std,1.154513e+08,5710.284645,970.004098,0.260807,1.844069,40.209250,2.323999,9.087564,6.145197,0.077107,0.080412,0.059172,0.050837,0.0,6.729773
min,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,5.000000e+02,9.000000,425.000000,0.000000,2.000000,6.000000,2.000000,1.000000,3.666667,0.047619,0.017627,0.000000,0.034483,0.0,0.000000
50%,1.300000e+03,29.000000,1057.000000,0.000000,3.000000,12.000000,3.000000,2.000000,5.400000,0.076923,0.033333,0.000000,0.057471,0.0,0.000000
75%,7.000000e+03,89.000000,2217.000000,0.149890,4.000000,26.000000,5.000000,4.000000,8.300000,0.125000,0.054945,0.050000,0.083333,0.0,0.000000
max,7.950101e+10,204429.000000,3066.000000,1.000000,21.000000,711.000000,9.000000,352.000000,273.000000,1.000000,1.000000,0.541667,1.000000,0.0,284.000000


Lets see for example how CAPS only ads reflect deal_probability:

In [460]:
train[train['deal_probability']>0.8].corr()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char
price,1.000000,-0.000266,0.002934,0.004336,-0.005562,0.001021,-0.007966,0.000674,0.001740,0.007866,-0.003255,-0.001340,-0.002660,NaN,0.001217
item_seq_number,-0.000266,1.000000,-0.000143,-0.012759,0.040724,0.041567,-0.017632,0.072395,-0.010801,0.008722,0.016795,0.035264,0.056770,NaN,0.084514
image_top_1,0.002934,-0.000143,1.000000,0.386418,0.035300,0.056882,-0.378917,0.070457,-0.007878,0.015810,0.036581,0.036132,0.010822,NaN,0.084812
deal_probability,0.004336,-0.012759,0.386418,1.000000,0.058118,0.113206,-0.043237,0.127753,0.013090,-0.102270,0.041179,0.047156,0.077806,NaN,0.127259
title_word_count,-0.005562,0.040724,0.035300,0.058118,1.000000,0.273613,0.006030,0.236083,0.074443,-0.348246,0.003534,0.278557,0.079870,NaN,0.193051
description_word_count,0.001021,0.041567,0.056882,0.113206,0.273613,1.000000,-0.060254,0.856418,0.127532,-0.036281,-0.021583,0.083106,0.195044,NaN,0.755139
merged_params_word_count,-0.007966,-0.017632,-0.378917,-0.043237,0.006030,-0.060254,1.000000,-0.047927,-0.014266,-0.053178,-0.008453,0.054188,0.017164,NaN,-0.055733
description_sentence_count,0.000674,0.072395,0.070457,0.127753,0.236083,0.856418,-0.047927,1.000000,-0.157516,-0.017857,0.043760,0.083753,0.332053,NaN,0.920547
description_words/sentence_ratio,0.001740,-0.010801,-0.007878,0.013090,0.074443,0.127532,-0.014266,-0.157516,1.000000,-0.073589,-0.156332,0.008565,-0.184425,NaN,-0.126007
title_capital_letters_ratio,0.007866,0.008722,0.015810,-0.102270,-0.348246,-0.036281,-0.053178,-0.017857,-0.073589,1.000000,0.040324,-0.144382,-0.000286,NaN,0.000693


In [461]:
train[train['title_capital_letters_ratio'] < 0.1].corr()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char
price,1.000000,0.078363,0.052813,-0.013584,0.074562,0.063112,-0.030265,0.041064,0.022282,-0.086798,-0.005669,0.098698,0.011326,NaN,0.016600
item_seq_number,0.078363,1.000000,0.100495,-0.038557,0.131823,0.121401,-0.047399,0.115724,-0.001143,-0.167618,0.016771,0.219966,0.065819,NaN,0.081394
image_top_1,0.052813,0.100495,1.000000,0.187711,0.225096,0.186483,-0.554048,0.163235,0.047634,-0.172240,0.051578,0.229923,0.051788,NaN,0.142201
deal_probability,-0.013584,-0.038557,0.187711,1.000000,-0.004865,-0.002850,-0.117429,-0.014087,0.033433,-0.019637,-0.000794,0.009878,-0.009379,NaN,-0.019696
title_word_count,0.074562,0.131823,0.225096,-0.004865,1.000000,0.284235,-0.142808,0.231022,0.093226,-0.560147,0.008072,0.457411,0.102381,NaN,0.165971
description_word_count,0.063112,0.121401,0.186483,-0.002850,0.284235,1.000000,-0.157407,0.866002,0.102523,-0.162974,0.028635,0.206356,0.266991,NaN,0.740767
merged_params_word_count,-0.030265,-0.047399,-0.554048,-0.117429,-0.142808,-0.157407,1.000000,-0.140317,-0.033047,0.057530,-0.035917,-0.053787,-0.050658,NaN,-0.133075
description_sentence_count,0.041064,0.115724,0.163235,-0.014087,0.231022,0.866002,-0.140317,1.000000,-0.160957,-0.106639,0.081650,0.155642,0.398407,NaN,0.924789
description_words/sentence_ratio,0.022282,-0.001143,0.047634,0.033433,0.093226,0.102523,-0.033047,-0.160957,1.000000,-0.087997,-0.138379,0.069727,-0.178563,NaN,-0.137088
title_capital_letters_ratio,-0.086798,-0.167618,-0.172240,-0.019637,-0.560147,-0.162974,0.057530,-0.106639,-0.087997,1.000000,0.022594,-0.572017,-0.028903,NaN,-0.036329


We can see a (very) slight correlation between description_len, and a (slight) negative correlation between deal_probability and "title capital letters ratio".

Iterestingly we also see a negative correlation between "title capital letters ratio" and "item_seq_num" and "image_top_1".

## Part of speech tagging for text fields

In this section we use NLTK's POS russian tagger.
For details:

https://www.nltk.org/book/ch05.html

https://www.nltk.org/_modules/nltk/tag.html

The russian tagger rags sentences using the Russian National Corpus tagset:

http://www.ruscorpora.ru/en/corpora-morph.html

Here are some of the most important tags:
* S – noun
* A – adjective 
* NUM – numeral 
* A-NUM – numeral adjective 
* V – verb 
* ADV – adverb 

###### Caution! It takes a few hours for those to finish.
Let's tokenize and tag our titles and descriptions.

In [462]:
train["tagged_title"] = train.apply(lambda r : nltk.pos_tag(nltk.word_tokenize(r["title"]), lang="rus"), axis = 1)
train["tagged_description"] = train.apply(lambda r : nltk.pos_tag(nltk.word_tokenize(r["description"]), lang="rus"), axis = 1)

In [463]:
# train.to_pickle(os.path.join(DATA_PATH, 'tagged_train_data.pkl'))

In [464]:
# train.iloc[0:0]
# train = pd.read_pickle(os.path.join(DATA_PATH, 'tagged_train_data.pkl'))

How many adjectives (descriptions) there are in a sentence? How many nouns (products)? What are the ratio of those w.r.t title/description length?

In [465]:
stopwords = set()
import codecs
with codecs.open(os.path.join(DATA_PATH, 'stopwords_ru.txt'), encoding='cp1251') as ins:
    for w in ins:
        word = w.strip("\r\n")
        word = word.strip("\n")
        stopwords.add(word)
        stopwords.add(word.upper())
        stopwords.add(word[0].upper() + word[1:])         

In [466]:
non_letter = re.compile("[\W]")
def count_tokens(sentence, tag):
    res = 0
    for p in sentence:
        if (p[0] in stopwords):
            continue
        if tag.match(p[1]) and (not non_letter.match(p[0])):
            res += 1
    return res

for att in ["title", "description"]:
    train[att+"_num_adj"] = train.apply(lambda r : count_tokens(r["tagged_"+att], re.compile("A(=*)?")), axis = 1)
    train[att+"_num_nouns"] = train.apply(lambda r : count_tokens(r["tagged_"+att], re.compile("(S|NONLEX)")), axis = 1)
    train[att+"_adj_to_len_ratio"] = train[att+"_num_adj"] / train[att+"_word_count"]
    train[att+"_noun_to_len_ratio"] = train[att+"_num_nouns"] / train[att+"_word_count"]

Now we want to find *the* product that is being sold. We do this by picking the first two non-stop-words nouns from the title and the first three non-stop-words from a noun phrase from the description.
We save the words also in a stemmed way.

In [467]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [468]:
def find_three_tagged(s, tag):
    pos = [None,None,None]
    i = 0
    found = 0
    while ((found < 3) & (i < len(s))):
        if (s[i][0] not in stopwords) and tag.match(s[i][1]):
            pos[found] = s[i][0]
            found += 1
        i += 1
    return pos

In [469]:
for att in ["title", "description"]:
    nouns = train.apply(lambda r : find_three_tagged(r["tagged_"+att], re.compile("S")), axis = 1)
    adjs = train.apply(lambda r : find_three_tagged(r["tagged_"+att], re.compile("A(=*)?")), axis = 1)

In [470]:
def stem(s):
    if pd.isnull(s) or len(s) == 0:
        return None
    return stemmer.stem(s.lower())

num = ["_first", "_second", "_third"]
for att in ["title", "description"]:
    for i in range(3):
        train[att+num[i]+"_noun"] = nouns.apply(lambda r: r[i])
        train[att+num[i]+"_noun_stemmed"] = nouns.apply(lambda r: stem(r[i]))

At last we want to learn about the condition of the product being sold. We do the same thing for adjectives.

In [471]:
for att in ["title", "description"]:
    for i in range(3):
        train[att+num[i]+"_adj"] = adjs.apply(lambda r: r[i])
        train[att+num[i]+"_adj_stemmed"] = adjs.apply(lambda r: stem(r[i]))

In [472]:
train.describe()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char,title_num_adj,title_num_nouns,title_adj_to_len_ratio,title_noun_to_len_ratio,description_num_adj,description_num_nouns,description_adj_to_len_ratio,description_noun_to_len_ratio
count,4.747930e+05,503424.000000,465718.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,464619.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.0,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,464619.000000,464619.000000
mean,4.270890e+05,757.584619,1242.017173,0.139364,3.324291,25.126468,3.590051,4.678508,7.050668,0.097784,0.048276,0.033604,0.062609,0.0,1.885049,0.495127,2.073697,0.150808,0.673930,4.081899,11.400466,0.192059,0.465616
std,1.154513e+08,5710.284645,970.004098,0.260807,1.844069,40.209250,2.323999,9.087564,6.145197,0.077107,0.080412,0.059172,0.050837,0.0,6.729773,0.643708,1.160062,0.207074,0.254442,6.357169,18.408161,0.151846,0.171815
min,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000e+02,9.000000,425.000000,0.000000,2.000000,6.000000,2.000000,1.000000,3.666667,0.047619,0.017627,0.000000,0.034483,0.0,0.000000,0.000000,1.000000,0.000000,0.500000,1.000000,3.000000,0.100000,0.375000
50%,1.300000e+03,29.000000,1057.000000,0.000000,3.000000,12.000000,3.000000,2.000000,5.400000,0.076923,0.033333,0.000000,0.057471,0.0,0.000000,0.000000,2.000000,0.000000,0.666667,2.000000,6.000000,0.166667,0.454545
75%,7.000000e+03,89.000000,2217.000000,0.149890,4.000000,26.000000,5.000000,4.000000,8.300000,0.125000,0.054945,0.050000,0.083333,0.0,0.000000,1.000000,3.000000,0.250000,1.000000,5.000000,12.000000,0.250000,0.538462
max,7.950101e+10,204429.000000,3066.000000,1.000000,21.000000,711.000000,9.000000,352.000000,273.000000,1.000000,1.000000,0.541667,1.000000,0.0,284.000000,5.000000,11.000000,3.000000,7.000000,113.000000,393.000000,5.000000,10.000000


In [473]:
train.head(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,merged_params,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char,tagged_title,tagged_description,title_num_adj,title_num_nouns,title_adj_to_len_ratio,title_noun_to_len_ratio,description_num_adj,description_num_nouns,description_adj_to_len_ratio,description_noun_to_len_ratio,title_first_noun,title_first_noun_stemmed,title_second_noun,title_second_noun_stemmed,title_third_noun,title_third_noun_stemmed,description_first_noun,description_first_noun_stemmed,description_second_noun,description_second_noun_stemmed,description_third_noun,description_third_noun_stemmed,title_first_adj,title_first_adj_stemmed,title_second_adj,title_second_adj_stemmed,title_third_adj,title_third_adj_stemmed,description_first_adj,description_first_adj_stemmed,description_second_adj,description_second_adj_stemmed,description_third_adj,description_third_adj_stemmed
1000000,cf348255c0c1,4e3cfbb60abc,Волгоградская область,Волгоград,Для дома и дачи,Мебель и интерьер,Шкафы и комоды,NaN,NaN,Витрина-шкаф стеклянная импортная,Б/у хорош.,5500.0,77,2017-03-19,Company,326215e9b350e828d4bf440c7c7d92018ad1d808b3d45a...,1511.0,0.86521,Шкафы и комоды,3,2,3,1,2.000000,0.032258,0.111111,0.032258,0.222222,0,0,"[(Витрина-шкаф, ADV), (стеклянная, A=f), (импо...","[(Б/у, S), (хорош, A=m), (., NONLEX)]",3,0,1.000000,0.0,1,1,0.500000,0.500000,Б/у,б/у,None,None,None,None,Б/у,б/у,None,None,None,None,хорош,хорош,None,None,None,None,хорош,хорош,None,None,None,None
1000001,cac8c3840cdc,bc454d4dc63c,Ставропольский край,Ставрополь,Личные вещи,Товары для детей и игрушки,Игрушки,NaN,NaN,Неваляшеа,Неваляшка новая 28 см,400.0,49,2017-03-23,Private,ae12ed2c0db0226919f7a9ff2512e26104e78e8722fbcd...,703.0,0.00000,Игрушки,1,4,1,1,4.000000,0.111111,0.055556,0.000000,0.000000,0,0,"[(Неваляшеа, S)]","[(Неваляшка, S), (новая, A=f), (28, NUM=ciph),...",0,1,0.000000,1.0,1,2,0.250000,0.500000,Неваляшка,неваляшк,см,см,None,None,Неваляшка,неваляшк,см,см,None,None,новая,нов,None,None,None,None,новая,нов,None,None,None,None
1000002,008bb9e305b5,6d04035370f7,Нижегородская область,Нижний Новгород,Бытовая электроника,Товары для компьютера,Комплектующие,Видеокарты,NaN,Видеокарта asus geforce 210 (Новая),Интерфейс: PCI-E x16/\nТип: nVidia GeForce 210...,1000.0,4,2017-03-23,Private,0fd0580ee76984f30d9d498f17dfe13b289e1d7c95f608...,2865.0,0.76786,Комплектующие Видеокарты,5,55,2,15,3.666667,0.064516,0.126154,0.064516,0.141538,0,14,"[(Видеокарта, S), (asus, NONLEX), (geforce, NO...","[(Интерфейс, S), (:, NONLEX), (PCI-E, NONLEX),...",1,3,0.200000,0.6,4,38,0.072727,0.690909,Интерфейс,интерфейс,Тип,тип,Частота,частот,Интерфейс,интерфейс,Тип,тип,Частота,частот,шейдерного,шейдерн,Эффективная,эффективн,Максимальное,максимальн,шейдерного,шейдерн,Эффективная,эффективн,Максимальное,максимальн
1000003,4b05d6d213ed,68a3f4a6c928,Краснодарский край,Краснодар,Личные вещи,Часы и украшения,Ювелирные изделия,NaN,NaN,Кольцо из красного золота 585 пр,"Золотое кольцо 585 пр./\n /\nВес - 5,61 гр./\n...",12000.0,12276,2017-03-22,Shop,2a413ffdd5d746c54d51c2fd6fb0c4fb924a4010210278...,25.0,0.00000,Ювелирные изделия,6,35,2,15,2.333333,0.037037,0.042857,0.000000,0.235714,0,12,"[(Кольцо, S), (из, PR), (красного, A=n), (золо...","[(Золотое, A=n), (кольцо, S), (585, NUM=ciph),...",1,3,0.166667,0.5,1,14,0.028571,0.400000,кольцо,кольц,пр./,пр./,Вес,вес,кольцо,кольц,пр./,пр./,Вес,вес,Золотое,золот,None,None,None,None,Золотое,золот,None,None,None,None
1000004,1fc917874c95,fe44d46f5b5f,Нижегородская область,Нижний Новгород,Личные вещи,Детская одежда и обувь,Для мальчиков,Другое,NaN,Праздничный костюм,продам

In [474]:
# train.to_pickle(os.path.join(DATA_PATH, 'tagged_train_data_plus.pkl'))

How many unique (distinct) nouns we have? How many distinct adjs we have?

In [475]:
print("Num of distinct nouns: "
      ,len(set(train["title_first_noun"].tolist()) | set(train["title_second_noun"].tolist()) | set(train["title_third_noun"].tolist())))
print("Num of distinct stemmed nouns: "
      ,len(set(train["title_first_noun_stemmed"].tolist()) | set(train["title_second_noun_stemmed"].tolist()) | set(train["title_third_noun_stemmed"].tolist())))
print("Num of distinct adjs: "
      ,len(set(train["title_first_adj"].tolist()) | set(train["title_second_adj"].tolist()) | set(train["title_third_adj"].tolist())))
print("Num of distinct stemmed adjs: "
      ,len(set(train["title_first_adj_stemmed"].tolist()) | set(train["title_second_adj_stemmed"].tolist()) | set(train["title_third_adj_stemmed"].tolist())))

Num of distinct nouns:  123431
Num of distinct stemmed nouns:  77403
Num of distinct adjs:  76854
Num of distinct stemmed adjs:  38892


## Sentiment analysis

Let's strat with a simple out-of-the box sentiment analysis provedid by Poliglot. The scale of the words’ "polarity" consisted of three degrees: +1 for positive words, and -1 for negatives words (Same works for sentences). Neutral words will have a score of 0.

http://polyglot.readthedocs.io/en/latest/Sentiment.html

In [476]:
def get_sentiment(s):
    if pd.isnull(s) or len(s) == 0:
        return None
    t = Text(s)
    t.language = 'ru'
    return t.polarity

train['title_sentiment'] = train.apply(lambda row: get_sentiment(row['title']), axis=1)

In [477]:
train['description_sentiment'] = train.apply(lambda row: get_sentiment(row['description']), axis=1)

## Lets see what we've got by now

In [478]:
display(train[:10])

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability,merged_params,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char,tagged_title,tagged_description,title_num_adj,title_num_nouns,title_adj_to_len_ratio,title_noun_to_len_ratio,description_num_adj,description_num_nouns,description_adj_to_len_ratio,description_noun_to_len_ratio,title_first_noun,title_first_noun_stemmed,title_second_noun,title_second_noun_stemmed,title_third_noun,title_third_noun_stemmed,description_first_noun,description_first_noun_stemmed,description_second_noun,description_second_noun_stemmed,description_third_noun,description_third_noun_stemmed,title_first_adj,title_first_adj_stemmed,title_second_adj,title_second_adj_stemmed,title_third_adj,title_third_adj_stemmed,description_first_adj,description_first_adj_stemmed,description_second_adj,description_second_adj_stemmed,description_third_adj,description_third_adj_stemmed,title_sentiment,description_sentiment
1000000,cf348255c0c1,4e3cfbb60abc,Волгоградская область,Волгоград,Для дома и дачи,Мебель и интерьер,Шкафы и комоды,NaN,NaN,Витрина-шкаф стеклянная импортная,Б/у хорош.,5500.0,77,2017-03-19,Company,326215e9b350e828d4bf440c7c7d92018ad1d808b3d45a...,1511.0,0.86521,Шкафы и комоды,3,2,3,1,2.000000,0.032258,0.111111,0.032258,0.222222,0,0,"[(Витрина-шкаф, ADV), (стеклянная, A=f), (импо...","[(Б/у, S), (хорош, A=m), (., NONLEX)]",3,0,1.000000,0.000000,1,1,0.500000,0.500000,Б/у,б/у,None,None,None,None,Б/у,б/у,None,None,None,None,хорош,хорош,None,None,None,None,хорош,хорош,None,None,None,None,0.0,0.000000
1000001,cac8c3840cdc,bc454d4dc63c,Ставропольский край,Ставрополь,Личные вещи,Товары для детей и игрушки,Игрушки,NaN,NaN,Неваляшеа,Неваляшка новая 28 см,400.0,49,2017-03-23,Private,ae12ed2c0db0226919f7a9ff2512e26104e78e8722fbcd...,703.0,0.00000,Игрушки,1,4,1,1,4.000000,0.111111,0.055556,0.000000,0.000000,0,0,"[(Неваляшеа, S)]","[(Неваляшка, S), (новая, A=f), (28, NUM=ciph),...",0,1,0.000000,1.000000,1,2,0.250000,0.500000,Неваляшка,неваляшк,см,см,None,None,Неваляшка,неваляшк,см,см,None,None,новая,нов,None,None,None,None,новая,нов,None,None,None,None,0.0,0.000000
1000002,008bb9e305b5,6d04035370f7,Нижегородская область,Нижний Новгород,Бытовая электроника,Товары для компьютера,Комплектующие,Видеокарты,NaN,Видеокарта asus geforce 210 (Новая),Интерфейс: PCI-E x16/\nТип: nVidia GeForce 210...,1000.0,4,2017-03-23,Private,0fd0580ee76984f30d9d498f17dfe13b289e1d7c95f608...,2865.0,0.76786,Комплектующие Видеокарты,5,55,2,15,3.666667,0.064516,0.126154,0.064516,0.141538,0,14,"[(Видеокарта, S), (asus, NONLEX), (geforce, NO...","[(Интерфейс, S), (:, NONLEX), (PCI-E, NONLEX),...",1,3,0.200000,0.600000,4,38,0.072727,0.690909,Интерфейс,интерфейс,Тип,тип,Частота,частот,Интерфейс,интерфейс,Тип,тип,Частота,частот,шейдерного,шейдерн,Эффективная,эффективн,Максимальное,максимальн,шейдерного,шейдерн,Эффективная,эффективн,Максимальное,максимальн,0.0,0.000000
1000003,4b05d6d213ed,68a3f4a6c928,Краснодарский край,Краснодар,Личные вещи,Часы и украшения,Ювелирные изделия,NaN,NaN,Кольцо из красного золота 585 пр,"Золотое кольцо 585 пр./\n /\nВес - 5,61 гр./\n...",12000.0,12276,2017-03-22,Shop,2a413ffdd5d746c54d51c2fd6fb0c4fb924a4010210278...,25.0,0.00000,Ювелирные изделия,6,35,2,15,2.333333,0.037037,0.042857,0.000000,0.235714,0,12,"[(Кольцо, S), (из, PR), (красного, A=n), (золо...","[(Золотое, A=n), (кольцо, S), (585, NUM=ciph),...",1,3,0.166667,0.500000,1,14,0.028571,0.400000,кольцо,кольц,пр./,пр./,Вес,вес,кольцо,кольц,пр./,пр./,Вес,вес,Золотое,золот,None,None,None,None,Золотое,золот,None,None,None,None,0.0,0.000000
1000004,1fc917874c95,fe44d46f5b5f,Нижегородская

In [479]:
train.describe()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char,title_num_adj,title_num_nouns,title_adj_to_len_ratio,title_noun_to_len_ratio,description_num_adj,description_num_nouns,description_adj_to_len_ratio,description_noun_to_len_ratio,title_sentiment,description_sentiment
count,4.747930e+05,503424.000000,465718.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,464619.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.0,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,503424.000000,464619.000000,464619.000000,503424.000000,464619.000000
mean,4.270890e+05,757.584619,1242.017173,0.139364,3.324291,25.126468,3.590051,4.678508,7.050668,0.097784,0.048276,0.033604,0.062609,0.0,1.885049,0.495127,2.073697,0.150808,0.673930,4.081899,11.400466,0.192059,0.465616,0.014572,0.194744
std,1.154513e+08,5710.284645,970.004098,0.260807,1.844069,40.209250,2.323999,9.087564,6.145197,0.077107,0.080412,0.059172,0.050837,0.0,6.729773,0.643708,1.160062,0.207074,0.254442,6.357169,18.408161,0.151846,0.171815,0.188864,0.559454
min,0.000000e+00,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,5.000000e+02,9.000000,425.000000,0.000000,2.000000,6.000000,2.000000,1.000000,3.666667,0.047619,0.017627,0.000000,0.034483,0.0,0.000000,0.000000,1.000000,0.000000,0.500000,1.000000,3.000000,0.100000,0.375000,0.000000,0.000000
50%,1.300000e+03,29.000000,1057.000000,0.000000,3.000000,12.000000,3.000000,2.000000,5.400000,0.076923,0.033333,0.000000,0.057471,0.0,0.000000,0.000000,2.000000,0.000000,0.666667,2.000000,6.000000,0.166667,0.454545,0.000000,0.000000
75%,7.000000e+03,89.000000,2217.000000,0.149890,4.000000,26.000000,5.000000,4.000000,8.300000,0.125000,0.054945,0.050000,0.083333,0.0,0.000000,1.000000,3.000000,0.250000,1.000000,5.000000,12.000000,0.250000,0.538462,0.000000,1.000000
max,7.950101e+10,204429.000000,3066.000000,1.000000,21.000000,711.000000,9.000000,352.000000,273.000000,1.000000,1.000000,0.541667,1.000000,0.0,284.000000,5.000000,11.000000,3.000000,7.000000,113.000000,393.000000,5.000000,10.000000,1.000000,1.000000


In [480]:
train.corr()

,price,item_seq_number,image_top_1,deal_probability,title_word_count,description_word_count,merged_params_word_count,description_sentence_count,description_words/sentence_ratio,title_capital_letters_ratio,description_capital_letters_ratio,title_non_regular_chars_ratio,description_non_regular_chars_ratio,title_num_of_newrow_char,description_num_of_newrow_char,title_num_adj,title_num_nouns,title_adj_to_len_ratio,title_noun_to_len_ratio,description_num_adj,description_num_nouns,description_adj_to_len_ratio,description_noun_to_len_ratio,title_sentiment,description_sentiment
price,1.000000,0.002455,0.036044,-0.001151,0.002909,0.001804,-0.001210,0.000754,0.001211,0.001053,-0.000647,0.006762,-0.000142,NaN,-0.000029,-0.000850,0.002789,-0.001474,-0.000528,0.003132,0.001384,0.000872,-0.001893,-0.000236,-0.003560
item_seq_number,0.002455,1.000000,0.091740,-0.036210,0.131466,0.115458,-0.054582,0.116948,-0.006115,-0.056809,0.022625,0.195412,0.084832,NaN,0.094925,0.027501,0.104503,-0.019549,-0.039002,0.122096,0.115229,-0.018437,0.004012,0.012026,0.028963
image_top_1,0.036044,0.091740,1.000000,0.190061,0.247144,0.186875,-0.558718,0.168786,0.044932,0.130933,0.084823,0.190820,0.086116,NaN,0.160674,-0.074632,0.256850,-0.148715,-0.026517,0.140894,0.201734,-0.136048,0.062464,0.000846,0.002188
deal_probability,-0.001151,-0.036210,0.190061,1.000000,0.016185,0.000063,-0.119630,-0.014501,0.046585,0.022133,0.002278,0.020539,-0.009140,NaN,-0.022087,-0.042181,0.014773,-0.045391,-0.016050,-0.001882,0.007021,-0.039578,0.027096,-0.006991,-0.020704
title_word_count,0.002909,0.131466,0.247144,0.016185,1.000000,0.312962,-0.168954,0.253858,0.085193,-0.302297,0.031146,0.437960,0.141117,NaN,0.198569,0.291957,0.816463,-0.016239,-0.355159,0.304913,0.314538,-0.082214,0.029625,0.065731,0.018266
description_word_count,0.001804,0.115458,0.186875,0.000063,0.312962,1.000000,-0.164985,0.852048,0.097486,-0.012688,0.041867,0.187462,0.294011,NaN,0.751576,0.079717,0.302204,-0.025142,-0.055807,0.898032,0.969458,-0.128939,-0.045775,0.023004,0.101146
merged_params_word_count,-0.001210,-0.054582,-0.558718,-0.119630,-0.168954,-0.164985,1.000000,-0.146614,-0.025113,-0.024942,-0.049991,-0.047660,-0.074127,NaN,-0.140058,0.040941,-0.178136,0.086243,0.017894,-0.124715,-0.169029,0.118573,-0.036478,-0.001860,0.001234
description_sentence_count,0.000754,0.116948,0.168786,-0.014501,0.253858,0.852048,-0.146614,1.000000,-0.148159,0.020329,0.091521,0.133150,0.410705,NaN,0.938552,0.069156,0.259122,-0.017759,-0.030620,0.692323,0.841392,-0.125590,-0.013953,0.024830,0.068849
description_words/sentence_ratio,0.001211,-0.006115,0.044932,0.046585,0.085193,0.097486,-0.025113,-0.148159,1.000000,-0.028009,-0.138989,0.075122,-0.176028,NaN,-0.128865,-0.000920,0.053113,-0.022109,-0.054823,0.127693,0.093252,-0.068001,-0.047225,-0.002022,0.064827
title_capital_letters_ratio,0.001053,-0.056809,0.130933,0.022133,-0.302297,-0.012688,-0.024942,0.020329,-0.028009,1.000000,0.080344,-0.142216,0.024623,NaN,0.056203,-0.400936,-0.083754,-0.329342,0.359278,-0.073778,0.001056,-0.059374,0.037258,-0.034078,-0.006860


In [481]:
train.to_csv(os.path.join(DATA_PATH, '4tagged_semanted_train_data.csv'))
train.to_pickle(os.path.join(DATA_PATH, '4tagged_semanted_train_data.pkl'))

In [482]:
reduced = train.drop(["title","description","tagged_title", "tagged_description"], axis=1)

In [483]:
reduced.to_csv(os.path.join(DATA_PATH, '4tagged_semanted_train_data_reduced.csv'))
reduced.to_pickle(os.path.join(DATA_PATH, '4tagged_semanted_train_data_reduced.pkl'))

# TF-IDF processing

In order to create textual features that we can feed into our descition tree models (LGBM) we need to engeneer a numeric feature(s) that represent each ad's text in a unique way.
We descided to use TF-IDF text vectorizaiotn on our data set (we view the text features of each ad as a "document" in the corpus). But by itself it is not enough because it outputs a vector which can be fed to a neural network, but not to an LGBM model. To generate numeric features we use SVD decomposition on the matrix of all those vectors to extract the projection of each document on the K(~30) most significant PCA components of this matrix.
Those projections represent the K "most important" (tf-idf-vector-space wise) features of the text and to some extent represent the information each ad's text contains.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing, model_selection, metrics

In [ ]:
train_df = pd.read_csv('/home/u14303/Avito/train.csv')
test_df = pd.read_csv('/home/u14303/Avito/test.csv')
train_df["description"].fillna("NA", inplace=True)
train_df["title"].fillna("NA", inplace=True)
test_df["description"].fillna("NA", inplace=True)
test_df["title"].fillna("NA", inplace=True)

In [ ]:
n_svd_components = 30

In [ ]:
def tfidf_main(train_df, test_df, col_name, n_comp):    
    ### TFIDF Vectorizer ###
    tfidf_vec = TfidfVectorizer(ngram_range=(1,2), preprocessor=stem, stop_words=stopwords)
    full_tfidf = tfidf_vec.fit_transform(train_df[col_name].values.tolist() + test_df[col_name].values.tolist())

    ### SVD Components ###
    svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
    svd_obj.fit(full_tfidf)

    # Train
    train_tfidf = tfidf_vec.transform(train_df[col_name].values.tolist())
    train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
    train_svd.columns = ['%s_svd_%s_ngram' % (col_name, i+1) for i in range(n_comp)]
    train_df = pd.concat([train_df, train_svd], axis=1)

    # Test
    test_tfidf = tfidf_vec.transform(test_df[col_name].values.tolist())
    test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    test_svd.columns = ['%s_svd_%s_ngram' % (col_name, i+1) for i in range(n_comp)]
    test_df = pd.concat([test_df, test_svd], axis=1)
    
    return train_df, test_df

In [ ]:
generated_col_names = []
for col_name in ['title', 'description']:
    train_df, test_df = tfidf_main(train_df, test_df, col_name, n_svd_components)
    generated_col_names += ['%s_svd_%s_ngram' % (col_name, i+1) for i in range(n_svd_components)]

In [ ]:
train_df.columns

In [ ]:
generated_col_names += ['item_id']

In [ ]:
train_df_save = train_df[generated_col_names]

In [ ]:
test_df_save = test_df[generated_col_names]

In [ ]:
# train_df_save.to_csv(os.path.join(DATA_PATH, 'train_tfidf_uni_bi_grams_data.csv'))
# test_df_save.to_csv(os.path.join(DATA_PATH, 'test_tfidf_uni_bi_grams_data.csv'))

In [ ]:
train_df_save.to_csv("/home/u14303/Avito/train_tfidf_uni_bi_grams_data2.csv.gz", compression='gzip', index=False)
test_df_save.to_csv("/home/u14303/Avito/test_tfidf_uni_bi_grams_data2.csv.gz", compression='gzip', index=False)

We preoceed with text preprocessing and vectorization in the neural network notebooks where the context of text vectorization is critical (unlike general feature engineering as presented here).